In [2]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
pd.__version__

'2.2.2'

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')

In [7]:
df_columns = jan_df.shape[1]
df_columns

20

In [8]:
jan_df['duration'] = jan_df.lpep_dropoff_datetime - jan_df.lpep_pickup_datetime
jan_df.duration = jan_df.duration.apply(lambda td: td.total_seconds() / 60)
jan_sd = jan_df['duration'].std()
# df = jan_df[(Jjn_df.duration >= 1) & (jan_df.duration <= 60)]
jan_filtered = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]
# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
jan_sd
# df[categorical] = df[categorical].astype(str)

69.28096981389236

In [9]:
fraction_remaining = len(jan_filtered) / len(jan_df)
fraction_remaining

0.9655140489746794

In [10]:
train_dicts = jan_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
feature_matrix = dv.fit_transform(train_dicts)

In [11]:
num_columns = feature_matrix.shape[1]

In [12]:
num_columns

2

In [13]:
target = jan_filtered['duration'].values
model = LinearRegression()
model.fit(feature_matrix, target)

LinearRegression()

In [14]:
train_predictions = model.predict(feature_matrix)
rmse_train = np.sqrt(mean_squared_error(target, train_predictions))

In [15]:
rmse_train

9.241043061957221

In [16]:
feb_df['duration'] = feb_df.lpep_dropoff_datetime - feb_df.lpep_pickup_datetime
feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)
feb_filtered = feb_df[(feb_df.duration >= 1) & (feb_df.duration <= 60)]
feb_filtered['PULocationID'] = feb_filtered['PULocationID'].astype(str)
feb_filtered['DOLocationID'] = feb_filtered['DOLocationID'].astype(str)
feb_data_dicts = feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
feb_feature_matrix = dv.transform(feb_data_dicts)

# Calculate the RMSE on the validation data
feb_target = feb_filtered['duration'].values
val_predictions = model.predict(feb_feature_matrix)
rmse_val = np.sqrt(mean_squared_error(feb_target, val_predictions))

# Print results
num_columns, rmse_train, rmse_val

/tmp/ipykernel_2615/1287206447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_filtered['PULocationID'] = feb_filtered['PULocationID'].astype(str)
/tmp/ipykernel_2615/1287206447.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_filtered['DOLocationID'] = feb_filtered['DOLocationID'].astype(str)


(2, 9.241043061957221, 9.741871514315305)

7.946010441252073